# *Search and Intermediation in Last Mile Shipping*
<br>
By **Ginha Kim** <br>
Structural Estimation Final Project, *Winter 2019*
***
This project implements structural estimation (Maximum Empirical Likelihood) of the Burdett and Judd (1983) non-sequential search model. Using the methodology proposed by Hong and Shum (2006), I estimate search cost distributions with only observed prices data. <br>

** Import necessary libraries **

In [ ]:
import numpy as np
import scipy
import scipy.stats as sts
from scipy.stats import skewnorm
import scipy.optimize as opt
import matplotlib.pyplot as plt
%matplotlib notebook

## 1.  Data

### 1.1 Import and Load Data

<br>
The data on observed prices is obtained from the third party logistics firm DAT.<br>
The datasets provides information on spot market and contract rates for dry-van full truck loads per ton-mile.<br>
I convert each into one-dimensional vectors of prices.

** 1.1.1 ** Before loading the actual data, I simulate prices using national averages.

In [ ]:
# Contract Rates Simulation

cr_avgs  = np.array([3.02, 2.55, 2.14, 2.35, 1.91, 1.89, 2.33])
mean_cr  = float(np.mean(cr_avgs))
std_cr   = float(np.std(cr_avgs))
dum_cr   = skewnorm.rvs(0.77, loc = mean_cr , scale = std_cr, size = 1000)

# Spot Market Rates Simulation

sr_avgs = np.array([2.08, 1.97, 2.14, 1.86, 1.71, 1.53])
mean_sr   = float(np.mean(sr_avgs))
std_sr    = float(np.std(sr_avgs))
dum_sr    = skewnorm.rvs(0.77, loc = mean_sr , scale = std_sr, size = 1000)

cont_prices = np.sort(dum_cr)
spot_prices = np.sort(dum_sr)

### 1.2 Summary Statistics

** 1.2.1 ** Simulated Contract Rates

In [ ]:
prices = cont_prices
mean_prices = np.mean(prices)
med_prices  = np.median(prices)
std_prices  = np.std(prices)
max_prices  = np.max(prices)
min_prices  = np.min(prices)
print('Dummy rates ($-per-ton/mile)')
print('Mean: ',mean_prices)
print('Median: ',med_prices)
print('Standard Deviation: ',std_prices)
print('Maximum: ',max_prices)
print('Minimum: ',min_prices)

** 1.2.2 ** Simulated Spot Rates

In [ ]:
prices = spot_prices
mean_prices = np.mean(prices)
med_prices  = np.median(prices)
std_prices  = np.std(prices)
max_prices  = np.max(prices)
min_prices  = np.min(prices)
print('Dummy rates ($-per-ton/mile)')
print('Mean: ',mean_prices)
print('Median: ',med_prices)
print('Standard Deviation: ',std_prices)
print('Maximum: ',max_prices)
print('Minimum: ',min_prices)

### 1.3 Histogram

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
n, bins, patches = plt.hist(cont_prices, 100, facecolor='blue' , stacked=True, label ='Simulated Contract Rates')
n, bins, patches = plt.hist(spot_prices, 100, facecolor='green', stacked=True, label ='Simulated Spot Rates')
plt.title('Simulated Prices Histogram') 
plt.xlabel('Dry-Van, Full-Truck-Load Rate per mile ($)')
plt.ylabel('% of observations in bin')
plt.legend()

## 2. The Non-Sequential Search Model - Burdett and Judd (1983) <br>
The following model takes from the non-seqeuntial search model specified by Burdett and Judd (1983) in their paper "Equilibrium Price Dispersion".

###   2.1 Equilibrium Conditions - Buyer side
*Buyer's Problem* <br>
Buyer's minimize the expected cost of acquiring the desired good by commiting to buying from the lowest priced seller over a fized number of searches (i.e. price quotations). The optimal number of searches $l^\ast \geq 1$ can be written as a function of per-search costs $c$ as follows: <br>
\begin{align}
    l^\ast(c) \equiv \underset{l>1}{arg\min} \quad c\cdot(l-1) + \int_{\underline{p}}^{\bar{p}} l \cdot p(1-F_p(p))^l-1 f(p) dp 
\end{align}

### 2.2 Equilibrium Conditions - Seller side
*Seller's Problem * <br>
Sellers maximize their profits earned from employing the mixed pricing strategy $F_p(\cdot)$. Let $r$ denote the common unit production cost and $\underline{p}, \bar{p}$ denote the lower and upper bound of the support of $F_p(\cdot)$, respectively. 
\par \noindent So the seller's profit from employing the mixed pricing strategy $F_p(\cdot)$ can be
written as follows:
\begin{align}
    \Pi(p) = (p-r)\bigg[\sum_{k=1}^{\infty}\tilde{q_k}k(1-F_p(p))^{k-1}\bigg] \qquad \text{for all  } p\in[\underline{p},\bar{p}]
\end{align}

## 3. Structural Estimation (MEL)<br>
The following estimation procedure follows the methodology developed by Hong and  Shum (2006) in their paper "Using price distributions to estimate search costs".

### 3.1 Construct the Empirical Distribution of Prices $\hat{F}_p(\cdot)$

**3.1.1** Index observations of prices in ascending order.

In [ ]:
prices = cont_prices 
#prices = spot_prices
n = prices.size

In [ ]:
prices = np.sort(prices)
#print(prices)

**3.1.2** Fix parameters $\underline{p}$ and $\bar{p}$ from values obtained from the data

In [ ]:
min_prices = prices[0]
max_prices = prices[n-1]

#print(min_prices,max_prices)

**3.1.3** Derive values of the empirical CDF of prices.

In [ ]:
def ecdf(x):
    
    '''
    Return empirical CDF of x
    
    '''

    sx = np.sort(x)
    
    ecdf = (1.0 + np.arange(len(sx)))/len(sx)
    
    return ecdf

In [ ]:
Fhat_prices = ecdf(prices)
#plt.plot(prices,Fhat_prices)
#print(Fhat_prices)

### 3.2 Construct moment conditions from equilibrium conditions<br>
The data consists of $n$ prices,  $p_i, i=1,...,n$. <br>
Consider a discrete price distribution with n points of support at $p_i, \quad i=1,...,n$.<br>
<br>
 \begin{align}
          F_p(p) = \sum_{i=1}^{n} \pi_{i} \mathbf{1}(p_i\leq p)\qquad i=1,...,n
  \end{align}
  <br>
 where $\pi_i$ denotes the probability weight at point $p_i$.<br>

Recall the indifference conditions we derived from the equilibrium restrictions described above: <br>
<br>
\begin{align}
    (\bar{p}-r)\tilde{q_1} = (p_i-r)\bigg[\sum_{k=1}^K \tilde{q_k}k(1-\hat{F}_p(p_i))^{k-1}\bigg] \qquad i = 1,...,n-1
\end{align}
<br>

For all $i=1,...,n-1$ and fixed $K$, the distrcete distritbution $F_p(\cdot)$ analogously yields:<br>
<br>
\begin{align}
    (\bar{p}-r)\tilde{q_1} = (p_i-r)\bigg[\sum_{k=1}^K \tilde{q_k}k\bigg(1-\bigg[\sum_{j=1}^{n} \pi_j \mathbf{1}(p_j\leq p_i)\bigg]\bigg)\bigg] \qquad i = 1,...,n-1 \quad \cdot\cdot\cdot\quad (1)
\end{align}
<br>

Let $\theta \equiv \{r,\tilde{q_1},....,\tilde{q}_K\}$ denote the unknown parameters to be estimated.

**3.2.1** Define necessary values including random sample of ECDF values $s_m \in [0,1], \quad m = 1,...,M, \quad M\geq K$

In [ ]:
K = 5
M = K + 3
dS  = np.sort(np.random.choice(Fhat_prices, size = M, replace = False))
print(dS)

** 3.2.2 ** Define functions that generates values for function variables<br>
<br>
\begin{align}
    \sum_{k=1}^K \tilde{q_k}k
\end{align}
<br>
\begin{align}
    \sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}
\end{align}
<br>

In [ ]:
def sum_qkk_gen(q):
    
    qkk = np.zeros(K)
    
    for i in range(K):
        
        qkk[i] = q[i] * (i+1)
        
    return np.sum(qkk)

def sum_qks_gen(q, s):
    
    qks = np.zeros(K)
    
    for i in range(K):
        
        qks[i] = q[i] * (i+1) * ((1 - s) ** i)
        
    return np.sum(qks)

** 3.2.3 ** Define $r$ as a function of $\underline{p},\bar{p},\text{ and } \mathbf{\tilde{q}} \equiv [\tilde{q}_1,...,\tilde{q}_K]$ <br>
<br>
\begin{align}
    r(\tilde{\mathbf{q}}) \equiv \frac{\underline{p}\cdot\big[\sum_{k=1}^K \tilde{q_k}k\big]-\bar{p}\cdot\tilde{q_1}}{\big[\sum_{k=1}^K \tilde{q_k}k\big]-\tilde{q_1}}
\end{align}
<br>
where $\underline{p}$ and $\bar{p}$ are given by the data and treated as known and nonstocastic parameters.
<br>

In [ ]:
def r_gen(prices, q):
    
    n = prices.size
    prices = np.sort(prices)
    min_p = prices[0]
    max_p = prices[n-1]
    
    numerator = (min_p * sum_qkk_gen(q)) - (max_p * q[0])
    
    denominator = sum_qkk_gen(q) - q[0]
    
    return numerator/denominator

**3.2.4** Define a funtion that generates $s_m$th quantile of $F_p(\cdot)$<br>
<br>
\begin{align}
& \Rightarrow F_p^{-1}(s_m) = r(\tilde{\mathbf{q}}) + \frac{(\bar{p}-r(\tilde{\mathbf{q}}))\tilde{q_1}}{\sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}} \equiv g_{s_m}(\tilde{\mathbf{q}})
\end{align}
<br>
\begin{align}
& \Rightarrow s_m\text{th quantile of } F_p(p) = g_{s_m}(\tilde{\mathbf{q}})
\end{align}
<br>

In [ ]:
def g_gen(prices, q, s):
    
    n = prices.size
    prices = np.sort(prices)
    min_p = prices[0]
    max_p = prices[n-1]

    num   = (max_p  - r_gen(prices, q)) * q[0]
    
    denom = sum_qks_gen(q, s)
    
    g = r_gen(prices, q) + (num / denom)
    
    return g

### 3.3 Maximum Empirical Likelihood - Formulation

The population quantile restriction can be rewritten as a population mean restriction:<br>
<br>
\begin{align*}
    E\bigg[\mathbf{1}\bigg(p_i \leq  r(\tilde{\mathbf{q}}) + \frac{(\bar{p}-r(\tilde{\mathbf{q}}))\tilde{q_1}}{\sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}} \bigg)-s_m\bigg] = 0, \qquad m = 1,...,M, \quad M \geq K
\end{align*}

The sample analogs of the population mean restrictions can be derived as follows:<br>
<br>
\begin{align}
    \sum_{j=1}^{n} \pi_j \cdot \bigg[\mathbf{1}\bigg(p_i \leq  r(\tilde{\mathbf{q}}) + \frac{(\bar{p}-r(\tilde{\mathbf{q}}))\tilde{q_1}}{\sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}} \bigg)-s_m \bigg] = 0  \quad \cdot\cdot\cdot\cdot\cdot \quad (2)
\end{align}
<br>

Then the empirical likelihood problem with respect to weights $\pi_i$ for $i=1,...,n$ and parameters $\theta$ can be written as follows:<br>
<br>
\begin{align}
    & \underset{\theta}{\max} \sum_{i=1}^{n} \log \pi_i \quad
    s.t. \quad \sum_{i=1}^{n} \pi_i = 1 
\end{align}
<br>
subject to the sample moment conditions in (1) and the summing-up condition.<br>

** 3.3.1 ** Define a function that calculates values of the indicator function<br>
<br>
\begin{align}
    \mathbf{1}\bigg(p_i \leq  p \bigg)
\end{align}
<br>

In [ ]:
def p_indc(pi, p):
    
    if pi < p:
        p_indc = 1
        
    elif pi == p:
        p_indc = 1
        
    elif pi > p:
        p_indc = 0
        
    return p_indc

** 3.3.2 ** Define a function that calculates the sum of probability weights $\pi_i$.

<br>
\begin{align}
    \sum_{i=1}^{n} \pi_i = 1 
\end{align}
<br>
$\pi_i$ denotes the probability weight at the point $p_i, i = 1,...,n$ where $p_i$'s are the $n$ points of support of the discrete price distibution.

In [ ]:
def weight_vec_gen(prices, Fhat):
    
    n = prices.size
    weights = np.zeros(n)
    
    for i in range(n):
        
        weights[0] = Fhat[0] - 0
        weights[i] = Fhat[i] - Fhat[i-1]
    
    return weights

#print(weight_vec_gen(prices,Fhat_prices))
#print(np.sum(weight_vec_gen(prices,Fhat_prices)))

** 3.3.4 ** Define a function that generates $F_p(\cdot)$.

<br>
\begin{align}
          F_p(p) = \sum_{i=1}^{n} \pi_{i} \mathbf{1}(p_i\leq p)\qquad i=1,...,n
\end{align}
<br>

In [ ]:
def F_p_gen(prices, p, Fhat):
    
    n = prices.size
    
    prob_vec = np.zeros(n)
    
    weights = weight_vec_gen(prices, Fhat)
    
    for i in range(n):
        
        pi = prices[i]
        
        wi = weights[i]
        
        prob_vec[i] = wi * p_indc(pi, p)
            
    prob = np.sum(prob_vec)

    return prob

#dindex = np.where(prices == dp)[0]
#print(dindex)
#print(dp)
#print(Fhat_prices[dindex])
#print(F_p_gen(prices, dp, Fhat_prices))

** 3.3.5 ** Generate $M$ moment conditions by iterating process over $M$ values of $s$ 

*Sanple Analog Moment Conditions:*
<br>
\begin{align}
    \sum_{j=1}^{n} \pi_j \cdot \bigg[\mathbf{1}\bigg(p_j \leq  r(\tilde{\mathbf{q}}) + \frac{(\bar{p}-r(\tilde{\mathbf{q}}))\tilde{q_1}}{\sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}} \bigg)-s_m \bigg] = 0  \quad \cdot\cdot\cdot\cdot\cdot \quad (2)
\end{align}
<br>

In [ ]:
def mom_gen(pi, q, S):
    
    M     = S.size
    mom_vec   = np.zeros(M)
    
    for m in range(M):
        
        sm = S[m]
        
        gq = g_gen(prices, q, sm)
        
        mom_vec[m] = p_indc(pi, gq) - sm
        
    return mom_vec
#print(mom_Fp_gen(dq, prices, dp, ds, Fhat_prices, min_prices, max_prices))

### 3.4 Maximum Empirical Likelihood - Estimation

** 3.4.1 ** Construct the objective function.

In [ ]:
def obj_func_gen(t, q, prices, S, Fhat):
    
    n     = prices.size
    min_p = prices[0]
    max_p = prices[n-1]
    
    weights = np.zeros(n)
    
    for i in range(n):
        
        pi = prices[i]
            
        mom = mom_gen(pi, q, S)

        weights[i] = 1 + np.dot(t,mom)
        
    return weights

** 3.4.2 ** Define the criterion function.

In [ ]:
def criterion(params, *args):
    
    t, q = params
    
    prices, S, Fhat = args
    
    ppf_vals     = obj_func_gen(t, q, prices, S, Fhat)
    log_ppf_vals = np.log(ppf_vals)
    log_like_val = log_ppf_vals.sum()
    neg_log_like = -log_like_val
    
    return neg_log_like

** 3.4.3 ** Construct the minimization problem

\begin{align}
    & \underset{\theta}{\max} \quad \underset{t}{\min}\quad \sum_{i=1}^{n}\log \bigg(1+ t'\bigg[\mathbf{1}\bigg(p_i \leq  r(\tilde{\mathbf{q}}) + \frac{(\bar{p}-r(\tilde{\mathbf{q}}))\tilde{q_1}}{\sum_{k=1}^K \tilde{q_k}k(1-s_m)^{k-1}} \bigg)-s_m \bigg]
    \bigg)
\end{align}

In [ ]:
# Initial Values
#q_init = np.zeros(K) + (1/K)
q_init = np.array([0.4,0.3,0.2,0.1,0])
t_init = np.zeros(M)
params_init = np.array([t_init, q_init])

# Boundaries 
#q_bounds = [(1e-10,1),(1e-10,1),(1e-10,1),(1e-10,1),(1e-10,1)]
#t_bounds = [(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None)]
#param_bounds = [(1e-10,1),(1e-10,None)]
#param_bounds    = (t_bounds, q_bounds)
#param_bounds= ((1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),(1e-10,None),
              # (1e-10,1),(1e-10,1),(1e-10,1),(1e-10,1),(1e-10,1))

# Constraints
#cons = ({'type': 'eq', 'fun' : lambda q: np.array([q[0]+q[1]+q[2]+q[3]+q[4]+q[5]-1])})

# MEL Arguments
mel_args = prices, dS, Fhat_prices
n = prices.size

#results  = opt.minimize(criterion, params_init, args = (mel_args), method='BFGS', options = {'gtol': 1e-7 * n, 'disp': True})

#results  = opt.minimize(criterion, params_init, args = (mel_args), method='L-BFGS-B', bounds = param_bounds, 
                        #options = {'gtol': 1e-7 * n, 'disp': True})
#results  = opt.minimize(criterion, params_init, args = (mel_args), method='SLSQP', bounds = param_bounds, constraints = cons, 
                        #options = {'gtol': 1e-7 * n, 'disp': True})
#results  = opt.minimize(criterion, params_init, args = (mel_args), method='CG', options = {'gtol': 1e-7 * n, 'disp': True})
#results  = opt.minimize(criterion, params_init, args = (mel_args), method='TNC', options = {'gtol': 1e-7 * n, 'disp': True})

print(results)
q_mele = np.array(results.x)[1]
r_mele = r_gen(spot_prices, q_mele)

## 4. Estimate the distribution of Search Costs

** 4.1** Define a function to recover the cut-off values at indiffence points of the distribution of search costs.<br>

\begin{align*}
    \tilde{q_k} = F_c(\Delta_{k-1})-F_c(\Delta_{k}) = \text{the proportion of buyers with $k$ price quotes}
\end{align*}

In [ ]:
def F_c_gen(q):
    
    K = q.size
    
    F_c_gen = np.zeros(K)
    
    qsum = []
    
    for i in range(K):
        
        F_c_gen[0] = 1 - q[0] 
        
        F_c_gen[i] = 1- np.sum(q[:i+1])
        
        F_c_gen[K-1] = 0
        
    return F_c_gen

print(F_c_gen(cdq))
print(F_c_gen(sdq))

** 4.2 ** Define a function that calcalates the expected value of the lowest among random draws

\begin{align*}
    Ep_{1:i} = \underline{p} + \int_{\underline{p}}^{\bar{p}} (1-F_p(p))^i dp
\end{align*}

In [ ]:
def E_p_gen(prices, Fhat, K):
    
    prices = np.sort(prices)
    n = prices.size
    min_p = prices[0]
    max_p = prices[n-1]
    E_p = np.zeros(K)
    
    for i in range(1, K+1):
        
        E_p[0]       = max_p
        iprice_draws = np.random.choice(prices, size = i, replace = False )
        price_draws  = np.sort(iprice_draws)
        lowest_price = price_draws[0]
        upper_index  = np.where(prices == lowest_price)[0] + 1
        lower_index  = np.where(prices == lowest_price)[0] - 1
        middle_index = np.where(prices == lowest_price)[0]
        prob_choice  = Fhat[upper_index] - Fhat[lower_index]
        prob_min     = ((1-Fhat[middle_index]) ** i)
        
        
        E_p[i-1]       = min_p + lowest_price * prob_choice * prob_min
        E_p[K-1] = min_p
        
    return E_p

print(E_p_gen(cont_prices, Fhat_cont, 5))
print(E_p_gen(spot_prices, Fhat_spot, 5))

** 4.3** Define a function that recovers indifference points from the cumulative distribution of search costs. <br>
<br>

\begin{align}
    \Delta_i \equiv Ep_{1:i}-Ep_{1:i+1} \qquad i = 1,2,...,
\end{align}

In [ ]:
def delta_gen(prices, Fhat, K):
    
    prices = np.sort(prices)
    n = prices.size
    min_p = prices[0]
    max_p = prices[n-1]
    
    delta = np.zeros(K)
    
    E_p = E_p_gen(prices, Fhat, K)
    
    for i in range(K-1):
        
        delta[0] = E_p[0] - E_p[1]
        delta[i] = E_p[i] - E_p[i+1]
        delta[i] = 100 * round(delta[i], 4)
        
    
    return delta

print(delta_gen(cont_prices, Fhat_cont, 5))
print(delta_gen(spot_prices, Fhat_spot, 5))

** 4.4 ** Plot the distribution of search costs

In [ ]:
cont_delta = delta_gen(cont_prices, Fhat_cont, 5)
spot_delta = delta_gen(spot_prices, Fhat_spot, 5)
X          = np.linspace(0,3,6)


F_cc   = F_c_gen(cdq)
F_cs   = F_c_gen(sdq)
FF_cc  = np.append(F_cc[::-1],np.ones(1))
FF_cs  = np.append(F_cs[::-1],np.ones(1))
print(FF_cc)
print(FF_cs)

In [ ]:
plt.plot(cont_delta, F_cc, linewidth=2, color ='c', label = 'Contracts $F_c(\cdot)$')
plt.plot(spot_delta, F_cs, linewidth=2, color ='g', label = 'Spot Market $F_c(\cdot)$')
#xcoords = np.sort(spot_delta)
#for xc in xcoords:
plt.axvline(x= cont_delta[0], linestyle='--',linewidth=0.7,color ='c')
plt.axvline(x= cont_delta[3], linestyle='--',linewidth=0.7,color ='c')
plt.axvline(x= spot_delta[0], linestyle='--',linewidth=0.7,color ='g')
plt.axvline(x= spot_delta[3], linestyle='--',linewidth=0.7,color ='g')
plt.annotate('$\Delta_1$', xy = (0.01+cont_delta[0],0.9), color = 'c',fontsize = 8)
plt.annotate('$\Delta_4$', xy = (0.01+cont_delta[3],0.9), color = 'c',fontsize = 8)
plt.annotate('$\Delta_1$', xy = (0.01+spot_delta[0],0.9), color = 'g',fontsize = 8)
plt.annotate('$\Delta_4$', xy = (0.01+spot_delta[3],0.9), color = 'g',fontsize = 8)
#xcoords = np.sort(cont_delta)
#for xc in xcoords:
    #plt.axvline(x=xc, linestyle='--')
#cp1, = plt.plot(cont_prices.mean(),0.5, marker='o', markersize=3, color="r")
#sp1, = plt.plot(spot_prices.mean(),0.5, marker='o', markersize=3, color="g")
#plt.plot(X, FF_cc, linewidth=2, color ='r', label = '$Contracts F_c(\cdot)$')
#plt.plot(X, FF_cs, linewidth=2, color ='g', label = '$Spot Market F_c(\cdot)$')
#plt.plot(prices, Fhat_prices, linewidth=2, color ='r', label = '$F_p(\cdot)$')
#p1, = plt.plot(prices[0],0.5, marker='o', markersize=3, color="red")
#p2, = plt.plot(mean_delta,0.5, marker='o', markersize=3, color="blue")
#p3, = plt.plot(prices[n-1],0.95, marker='o', markersize=3, color="red")
#plt.legend([p1,p2,p3], ["Min p","Max p","$\bar{c}$"])
#plt.xlim(-1e-10,3.8)
plt.ylim(0,1)
plt.title('Estimated Search Cost Distribution', fontsize = 12,fontweight = 'semibold',linespacing =2)
plt.xlabel('c = Cost per Search ($)')
plt.ylabel('Estimated Search Cost CDF')
plt.legend(loc='lower right')
plt.show()